In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:

data = pd.read_pickle('../create_SL_data/data_6months_levels.pkl')

data.dropna(inplace=True)

data

,patient_number,cluster_Basophils_size-diff%_pre_to_3h,cluster_Bcells_size-diff%_pre_to_3h,cluster_Doublets_size-diff%_pre_to_3h,cluster_Eosinophils_size-diff%_pre_to_3h,cluster_HSPCs_size-diff%_pre_to_3h,cluster_LinNeg_size-diff%_pre_to_3h,cluster_Monocytes_size-diff%_pre_to_3h,cluster_NKcells_size-diff%_pre_to_3h,cluster_NeutrophilCD16neg_size-diff%_pre_to_3h,...,q90_difference_pre_7d_cluster_NeutrophilsCD25_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_NeutrophilsHLA_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu1_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu2_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_PreNeu3_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_Tc_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_Th_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_pDC_158Gd_pSTAT3_Y705,BCRABL
0,01,35.135135,60.493827,32.682927,34.866434,90.239574,1120.000000,35.968601,57.288136,50.604844,...,0.000000,0.173798,0.228703,-0.634862,0.520689,0.445879,0.067349,0.119710,0.528887,0.0320
1,02,-30.165160,-15.625000,-13.513514,-45.743146,-39.000000,223.109244,-37.692571,-11.141304,-39.835804,...,0.000000,0.195533,0.319469,-0.824650,0.814652,0.649210,0.225261,0.701564,0.523819,0.0075
2,03,91.287290,60.059172,98.181818,25.997138,154.830918,295.501022,62.084592,-3.809524,40.061262,...,0.450780,0.013442,-0.006067,-2.014077,0.234621,0.509549,0.399494,0.270279,0.256670,0.0760
3,04,41.385171,88.732394,13.978495,24.392621,40.764331,97.575758,6.520173,25.301205,18.461538,...,0.125471,0.107060,-0.104685,-1.469164,0.578006,0.312356,0.016451,-0.092338,0.236907,0.6000
4,05,149.305556,196.078431,-3.225806,6.167401,30.000000,150.666667,98.670465,127.884615,22.222222,...,0.414612,0.588062,0.377221,-0.530531,0.544317,0.000000,0.104006,0.037562,0.000000,0.6300
5,06,7.061267,30.208333,46.153846,-26.833013,55.555556,110.256410,-16.031746,21.428571,10.078111,...,0.805617,0.126909,0.144516,-1.011029,0.615345,0.184870,0.058221,-0.048725,0.000000,3.3000
6,07,-7.244374,-0.273224,10.000000,6.280428,-31.139489,368.000000,0.289436,26.732673,22.421163,...,-0.723297,0.101798,0.123791,-0.386960,0.122739,0.275844,0.265686,0.621109,-0.836546,0.0054
7,08,-3.514676,21.551724,38.461538,-29.945629,46.462715,1622.844828,3.701299,-25.128205,-10.869139,...,0.751987,0.155500,0.112758,-1.508415,-0.032117,0.243146,0.096635,-0.365599,0.494612,0.1200
8,09,-21.217515,-32.635983,-71.604938,-5.039834,-32.792793,182.887701,-47.490668,-53.794940,4.181891,...,1.708393,0.515315,0.439256,-1.008428,0.613103,-0.129841,0.306030,0.615110,0.539467,0.0350
9,11,13.279730,20.719424,-11.538462,-10.527406,60.229885,264.285714,-9.048117,37.896825,7.837426,...,0.000000,0.329252,0.167228,-0.809707,0.860805,-0.124076,0.075963,-0.064593,-0.068443,0.0000


In [3]:
data['patient_number'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '11', '12',
       '13', '14', '15', '17', '18', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '32', '33', '34', '35', '36', '37', '38'],
      dtype=object)

In [4]:
X = data.drop(['BCRABL', 'patient_number'], axis=1)  
y = data['BCRABL']                                  

patient_numbers = data['patient_number']

## Leave one out nested cross validation

In [10]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import mean_squared_error

models = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 10, 50]}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
    },
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {} 
    }
}

feature_importances = {name: pd.DataFrame(np.zeros((X.shape[0], X.shape[1])), columns=X.columns) for name in models if name == 'RandomForest'}
scores = {name: {'mse_scores': [], 'rmse_scores': [], 'best_params': []} for name in models}

outer_cv = LeaveOneOut()
inner_cv = LeaveOneOut()

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, config in models.items():
        if config['params']:
            grid_search = GridSearchCV(config['model'], config['params'], cv=inner_cv, scoring='neg_mean_squared_error')
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            scores[name]['best_params'].append(grid_search.best_params_)
        else:
            best_model = config['model']
            best_model.fit(X_train, y_train)
            scores[name]['best_params'].append({})

        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        scores[name]['mse_scores'].append(mse)
        scores[name]['rmse_scores'].append(rmse)

        if name == 'RandomForest':
            feature_importances[name].iloc[fold_idx, :] = best_model.feature_importances_

    print(f'Fold {fold_idx + 1} completed')

results = {}
for name in models:
    average_mse = np.mean(scores[name]['mse_scores'])
    average_rmse = np.mean(scores[name]['rmse_scores'])
    results[name] = {
        'Average MSE': average_mse,
        'Average RMSE': average_rmse,
        'Best Parameters': scores[name]['best_params'][-1]
    }

    if name == 'RandomForest':
        mean_fi = feature_importances[name].mean(axis=0).sort_values(ascending=False)
        results[name]['Feature Importances'] = mean_fi

for name, result in results.items():
    print(f"{name} Results:")
    for key, value in result.items():
        print(f"{key}: {value}")


Fold 1 completed
Fold 2 completed
Fold 3 completed
Fold 4 completed
Fold 5 completed
Fold 6 completed
Fold 7 completed
Fold 8 completed
Fold 9 completed
Fold 10 completed
Fold 11 completed
Fold 12 completed
Fold 13 completed
Fold 14 completed
Fold 15 completed
Fold 16 completed
Fold 17 completed
Fold 18 completed
Fold 19 completed
Fold 20 completed
Fold 21 completed
Fold 22 completed
Fold 23 completed
Fold 24 completed
Fold 25 completed
Fold 26 completed
Fold 27 completed
Fold 28 completed
Fold 29 completed
Fold 30 completed
Fold 31 completed
Fold 32 completed
Fold 33 completed
RandomForest Results:
Average MSE: 1.7806477642843719
Average RMSE: 0.7777714013894144
Best Parameters: {'max_depth': 10, 'n_estimators': 100}
Feature Importances: 90th_151Eu_pSTAT3 S727_pre_cluster_Bcells                 7.787183e-02
90th_142Nd_cCaspase3_3h_cluster_NeutrophilCD16neg         4.013900e-02
90th_150Nd_pSTAT5 Y694_3h_cluster_PreNeu3                 2.809060e-02
90th_144Nd_pTyr_3h_cluster_Bcells     

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

model1 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model2 = KNeighborsRegressor(n_neighbors=10, weights='uniform')

model1.fit(X, y)
model2.fit(X, y)

KNeighborsRegressor(n_neighbors=10)

In [13]:
y.describe()

count    33.000000
mean      0.414785
std       1.211437
min       0.000000
25%       0.010000
50%       0.040000
75%       0.130000
max       6.320000
Name: BCRABL, dtype: float64

In [6]:
model1.score(X, y)

0.8396435156897418

In [7]:
model2.score(X, y)

0.04281529028380038

In [6]:
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(X, y)

RandomForestRegressor(max_depth=10, random_state=42)

In [41]:
pd.set_option('display.max_colwidth', None)

coefficients = model1.feature_importances_
feature_importance = pd.DataFrame(data={'Feature': X.columns, 'Importance': coefficients})

feature_importance['Absolute Importance'] = feature_importance['Importance'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Importance', ascending=False)

feature_importance.head(30)

,Feature,Importance,Absolute Importance
115,90th_151Eu_pSTAT3 S727_pre_cluster_Bcells,0.052471,0.052471
463,90th_158Gd_pSTAT3 Y705_3h_cluster_NKcells,0.047240,0.047240
965,q90_difference_pre_3h_cluster_PreNeu3_151Eu_pSTAT3_S727,0.044548,0.044548
395,90th_150Nd_pSTAT5 Y694_3h_cluster_PreNeu3,0.036459,0.036459
933,q90_difference_pre_3h_cluster_Doublets_176Yb_pS6_S240244,0.035186,0.035186
1166,q90_difference_pre_3h_cluster_NKcells_158Gd_pSTAT3_Y705,0.028475,0.028475
343,90th_143Nd_pCRKL Y207_3h_cluster_Bcells,0.026508,0.026508
806,90th_170Er_pSRC Y418_7d_cluster_NeutrophilCD16neg,0.020410,0.020410
705,90th_153Eu_pSTAT1 Y701_7d_cluster_Doublets,0.019655,0.019655
352,90th_143Nd_pCRKL Y207_3h_cluster_NeutrophilsCD25,0.019583,0.019583


In [7]:
data_pool2 = pd.read_pickle('../create_SL_data_pool2/data_6months_levels_an_pool2.pkl')

pool2_patients = pd.DataFrame(data_pool2['patient_number'])

In [8]:
X_test = data_pool2.drop(['BCRABL', 'patient_number'], axis=1) 
y_test = data_pool2['BCRABL']                                 

y_test

0    0.0200
1    2.1100
2    0.0020
3    1.0400
4    0.2220
5    0.0100
6    5.4000
7    0.0440
8    0.0026
Name: BCRABL, dtype: float64

In [9]:
model.predict(X_test)

array([0.313429, 0.58889 , 0.591404, 1.268072, 0.31502 , 0.559382,
       0.58215 , 0.576278, 0.68971 ])

In [11]:
pool2_patients['pred'] = model1.predict(X_test)
pool2_patients['month'] = 6
pool2_patients['patient_number'] = pool2_patients['patient_number'].astype(int)

pool2_patients

,patient_number,pred,month
0,39,0.313429,6
1,40,0.588890,6
2,41,0.591404,6
3,42,1.268072,6
4,44,0.315020,6
5,45,0.559382,6
6,54,0.582150,6
7,56,0.576278,6
8,57,0.689710,6


In [12]:
patient_numbers_pool2 = data_pool2['patient_number'].astype(int)

response = pd.read_csv('../response/responses_all.csv')
response_pool2 = response[response['patient_number'].isin(patient_numbers_pool2)]

response_pool2

,patient_number,patient_id,batch,month,BCRABL
237,39,4810_00009,5,3,0.070
238,39,4810_00009,5,6,0.020
239,39,4810_00009,5,9,0.020
240,39,4810_00009,5,12,0.007
241,39,4810_00009,5,15,0.009
...,...,...,...,...,...
295,57,4306_00004,6,9,0.025
296,57,4306_00004,6,12,NaN
297,57,4306_00004,6,15,NaN
298,57,4306_00004,6,17,0.007


In [32]:
merged_data = pd.merge(response_pool2, pool2_patients, on=['patient_number', 'month'], how='outer')
merged_data

,patient_number,patient_id,batch,month,BCRABL,pred
0,39,4810_00009,5,3,0.070,NaN
1,39,4810_00009,5,6,0.020,0.313429
2,39,4810_00009,5,9,0.020,NaN
3,39,4810_00009,5,12,0.007,NaN
4,39,4810_00009,5,15,0.009,NaN
...,...,...,...,...,...,...
58,57,4306_00004,6,9,0.025,NaN
59,57,4306_00004,6,12,NaN,NaN
60,57,4306_00004,6,15,NaN,NaN
61,57,4306_00004,6,17,0.007,NaN


In [33]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data['patient_number'].unique()):
    patient_data = merged_data[merged_data['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color) 
    ))
    
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False 
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log' 
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()



In [34]:
pool1_patients = pd.DataFrame(data['patient_number'])
pool1_patients['patient_number'] = pool1_patients['patient_number'].astype(int)
pool1_patients['pred'] = model1.predict(X)
pool1_patients['month'] = 6

pool1_patients

,patient_number,pred,month
0,1,0.797750,6
1,2,0.499181,6
2,3,0.175639,6
3,4,0.457185,6
4,5,0.445617,6
5,6,2.195575,6
6,7,0.244961,6
7,8,0.140152,6
8,9,0.117701,6
9,11,0.086099,6


In [35]:
merged_data = pd.merge(response, pool1_patients, on=['patient_number', 'month'], how='outer')
merged_data

merged_data['BCRABL'] = merged_data['BCRABL'].replace(0, 0.001)

In [36]:
merged_data_1 = merged_data[merged_data['batch'] == 1]
merged_data_2 = merged_data[merged_data['batch'] == 2]
merged_data_3 = merged_data[merged_data['batch'] == 3]
merged_data_4 = merged_data[merged_data['batch'] == 4]

In [37]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_1['patient_number'].unique()):
    patient_data = merged_data_1[merged_data_1['patient_number'] == patient]
    patient_color = colors[idx % len(colors)] 

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color) 
    ))
    
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values Batch 1',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()


In [38]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly 

fig = go.Figure()

for idx, patient in enumerate(merged_data_2['patient_number'].unique()):
    patient_data = merged_data_2[merged_data_2['patient_number'] == patient]
    patient_color = colors[idx % len(colors)] 

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    # Predicted value at 6 
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False 
        ))
        
       
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False 
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))


fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values Batch 2',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()


In [39]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly  

fig = go.Figure()

for idx, patient in enumerate(merged_data_3['patient_number'].unique()):
    patient_data = merged_data_3[merged_data_3['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color)  
    ))
    
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False 
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values Batch 3',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()


In [40]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

colors = px.colors.qualitative.Plotly

fig = go.Figure()

for idx, patient in enumerate(merged_data_4['patient_number'].unique()):
    patient_data = merged_data_4[merged_data_4['patient_number'] == patient]
    patient_color = colors[idx % len(colors)]  

    fig.add_trace(go.Scatter(
        x=patient_data['month'], y=patient_data['BCRABL'],
        mode='lines+markers', name=f'Patient {patient} Actual',
        line=dict(color=patient_color),  
        marker=dict(color=patient_color) 
    ))
    
    predicted_data = patient_data[patient_data['month'] == 6]
    if not predicted_data.empty:
        fig.add_trace(go.Scatter(
            x=[6], y=predicted_data['pred'],
            mode='markers', name=f'Patient {patient} Predicted',
            marker=dict(color=patient_color, size=10),  
            showlegend=False  
        ))
        
  
        before_pred = patient_data[patient_data['month'] < 6].tail(1)
        after_pred = patient_data[patient_data['month'] > 6].head(1)
        
        if not before_pred.empty:
            fig.add_trace(go.Scatter(
                x=[before_pred['month'].values[0], 6],
                y=[before_pred['BCRABL'].values[0], predicted_data['pred'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  #
            ))
        if not after_pred.empty:
            fig.add_trace(go.Scatter(
                x=[6, after_pred['month'].values[0]],
                y=[predicted_data['pred'].values[0], after_pred['BCRABL'].values[0]],
                mode='lines', line=dict(dash='dash', color=patient_color),
                showlegend=False  
            ))

fig.update_layout(
    title='Predicted BCR::ABL% at 6 months and actual values Batch 4',
    xaxis_title='Test Time (Months)',
    yaxis_title='BCR-ABL%',
    yaxis_type='log'  
)

fig.update_traces(connectgaps=True)

fig.add_hline(y=1)

fig.show()
